In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep='\t')
test_data = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep='\t')

In [ ]:
train_data.head()

In [ ]:
train_data.columns

In [ ]:
test_data.head()

In [ ]:
train_data['Sentiment'].value_counts()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.isna().sum()

In [ ]:
train_data = train_data.dropna()
print(train_data.shape)

In [ ]:
X = train_data.drop('Sentiment', axis=1)
X.head()

In [ ]:
y = train_data['Sentiment']

In [ ]:
from tensorflow.keras.layers import Embedding,Flatten,Dense,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
top_word = 5000

In [ ]:
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
messages=X.copy()
messages.reset_index(inplace=True)
messages.head()

In [ ]:
ps=PorterStemmer()
corpus_train=[]
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['Phrase'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus_train.append(review)

In [ ]:
corpus_train[0:10]

In [ ]:
messages['Phrase'][0:10]

In [ ]:
one_hot_rep=[one_hot(words,top_word) for words in corpus_train]
one_hot_rep[0]

In [ ]:
sent_length=20
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
embedded_docs[0]

In [ ]:
embedded_features=40
model=Sequential()
model.add(Embedding(top_word,embedded_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
X_final=np.asarray(embedded_docs)
y_final=np.asarray(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
train_model=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5)


In [ ]:
def test(X):
    X=X.fillna(0)
    messages=X.copy()
    messages.reset_index(inplace=True)
    corpus=[]
    for i in range(len(messages)):
        reviews=re.sub('[^a-zA-Z]',' ',str(messages['Phrase'][i]))
        reviews=reviews.lower()
        reviews=reviews.split()
        reviews=[ps.stem(word) for word in reviews if word not in stopwords.words('english')]
        reviews=' '.join(reviews)
        corpus.append(reviews)
    one_hot_rep=[one_hot(word,top_word)for word in corpus]
    embedded_docs = pad_sequences(one_hot_rep, padding = 'pre', maxlen = sent_length)
    X_final = np.array(embedded_docs)
    
    
    return X_final

In [ ]:
test_data_new= test(test_data)
test_data_new[0]


In [ ]:
pred=model.predict(test_data_new)
print(pred.shape)


In [ ]:
submission_data = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/submit.csv')
submission_data['Sentiment']=np.round(pred).astype('int')
submission_data.head()

In [ ]:
submission_data.to_csv('Submission.csv',index=False)